In [1]:
import pandas as pd
import os
import pyproj
import time
import numpy as np
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from pyprind import ProgBar

In [2]:
os.listdir()

['LAUSD_upper_3218.html',
 'LAUSD_no_charter_1.500.html',
 'LAUSD_no_charter_1.750.html',
 'LAUSD_upper_1207.html',
 'LAUSD_3621.html',
 'benson_challenge_MBF.ipynb',
 '.ipynb_checkpoints',
 'test.html',
 'LAUSD_upper_3621.html',
 'LAUSD_upper_2011.html',
 'LAUSD_4425.html',
 'LAUSD_1207.html',
 'LAUSD_upper_4023.html',
 'LAUSD.ipynb',
 'LAUSD_3218.html',
 'school list with coordinates_06302020.xlsx',
 'LAUSD_no_charter_2.000.html',
 'LAUSD_1609.html',
 'LAUSD_no_charter_2.750.html',
 'LAUSD_no_charter_3.000.html',
 'LAUSD_upper_4828.html',
 'LAUSD_no_charter_2.250.html',
 'LAUSD_no_charter_1.000.html',
 'LAUSD_no_charter_0.750.html',
 'LAUSD_upper_4425.html',
 'LAUSD_upper_1609.html',
 'LAUSD_no_charter_1.250.html',
 'LAUSD_upper_2816.html',
 'LAUSD_4023.html',
 'LAUSD_2011.html',
 'LAUSD_4828.html',
 'LAUSD_2414.html',
 'Bokeh-Mapping.ipynb',
 'LAUSD_no_charter_2.500.html',
 'LAUSD_upper_2414.html',
 'LAUSD_2816.html']

In [3]:
df = pd.read_excel('school list with coordinates_06302020.xlsx', sheet_name=1)

In [4]:
df.head()

,School Code,Campus Code,School cost center,Campus Cost Center,School Name,Local District,Physical LD,Board District,School Type Description,School Level Description,School Level Description (IAU),Lat,Long
0,5857,5857,1585701,1585701,107th Street Elementary,S,S,7.0,Elementary School,Elementary School,Elementary (K-5/ K-6),33.939372,-118.27266
1,5836,5836,1583601,1583601,109th Street Elementary,S,S,7.0,Elementary School,Elementary School,Elementary (K-5/ K-6),33.936944,-118.26088
2,7082,7082,1708201,1708201,10th Street Elementary,C,C,2.0,Elementary School,Elementary School,Primary (other),34.048370,-118.27348
3,5884,5884,1588401,1588401,112th Street Elementary,S,S,7.0,Elementary School,Elementary School,Elementary (K-5/ K-6),33.933190,-118.25270
4,5863,5863,1586301,1586301,116th Street Elementary,S,S,7.0,Elementary School,Elementary School,Elementary (K-5/ K-6),33.928201,-118.26208


In [5]:
df['Local District'].value_counts()

XR    227
W     144
C     141
E     135
S     130
NW    119
NE    109
XS      4
Name: Local District, dtype: int64

In [6]:
project_projection = pyproj.Proj("+init=EPSG:4326")  # wgs84
google_projection = pyproj.Proj("+init=EPSG:3857")  # default google projection

In [7]:
#while we're here, we also define the center of Los Angeles to map from
la_longitude = -118.15
la_latitude = 34.03
la_lon, la_lat = pyproj.transform(project_projection, google_projection, la_longitude, la_latitude)

In [8]:
def coord_transform(row):
    x, y = pyproj.transform(project_projection, google_projection, row['Long'], row['Lat'])
    return x, y
df['coords'] = df.apply(coord_transform, axis=1)

df[['X_COORD', 'Y_COORD']] = df['coords'].apply(pd.Series)

In [9]:
from bokeh.plotting import figure, show, output_file, reset_output, save
from bokeh.tile_providers import CARTODBPOSITRON
from bokeh.models import ColumnDataSource, ColorBar, HoverTool, LinearColorMapper, NumeralTickFormatter
from bokeh.io import export_png
from bokeh.io import show, output_notebook

output_notebook()

Loading BokehJS ...

In [10]:
#not a fan of the default bokeh palettes, so we make our own
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import rgb2hex
import seaborn as sns

dark = rgb2hex(sns.color_palette('Greens')[-2])
light = rgb2hex(sns.color_palette('Greens')[0])

cm = LinearSegmentedColormap.from_list('colors', [light, dark], N=100)
pal = []

for i in range(100):
    pal.append(rgb2hex(cm(i)))
    
low=0
high=100

In [11]:
# #changing this mask and title make it easy to make new plots
# #a little housekeeping
# reset_output()

# #add a hover tool for
# hover_tool = HoverTool(tooltips =[
#     ('Station', '@School Name')
# ])

# #associate the data with the plotting functions
# mapper = LinearColorMapper(palette=pal, low=low, high=high)
# data = ColumnDataSource(df)

# # and plot! center the figure in Manhattan, set the axis to mercator coordinates
# p = figure(x_range=(la_lon-65000, la_lon+25000), y_range=(la_lat-45000, la_lat+45000),
#            x_axis_type="mercator", y_axis_type="mercator",
#            plot_width=900, plot_height=600, title='')
# # add a circle for each station, sized and colored by the number of people
# p.circle(source=data, x='X_COORD', y='Y_COORD', alpha=0.8, line_color="black",
#         legend=False)
# #add open source geotiles
# p.add_tile(CARTODBPOSITRON)
# #hovertool, colorbar, and title aesthetics
# p.add_tools(hover_tool)
# color_bar = ColorBar(color_mapper=mapper, width=20,  location=(0,0), title='People',
#                     label_standoff=20, major_label_text_font_size='16pt',
#                     formatter= NumeralTickFormatter(format='0,0'))

# show(p)

https://stackoverflow.com/questions/15882202/minimum-number-of-circles-with-radius-r-to-cover-n-points

In [12]:
from collections import namedtuple
from itertools import product
from math import sqrt
from pprint import pprint as pp

Pt = namedtuple('Pt', 'x, y')
Cir = namedtuple('Cir', 'x, y, r')

def circles_from_p1p2r(p1, p2, r):
    'Following explanation at http://mathforum.org/library/drmath/view/53027.html'
    (x1, y1), (x2, y2) = p1, p2
    if p1 == p2:
        #raise ValueError('coincident points gives infinite number of Circles')
        return Cir(x1, y1, r), Cir(x1, y1, r)
    # delta x, delta y between points
    dx, dy = x2 - x1, y2 - y1
    # dist between points
    q = sqrt(dx**2 + dy**2)
    if q > 2.0*r:
        #raise ValueError('separation of points > diameter')
        return None, None
    # halfway point
    x3, y3 = (x1+x2)/2, (y1+y2)/2
    # distance along the mirror line
    d = sqrt(r**2-(q/2)**2)
    # One answer
    c1 = Cir(x = x3 - d*dy/q,
             y = y3 + d*dx/q,
             r = abs(r))
    # The other answer
    c2 = Cir(x = x3 + d*dy/q,
             y = y3 - d*dx/q,
             r = abs(r))
    return c1, c2

def covers(c, pt):
    return (c.x - pt.x)**2 + (c.y - pt.y)**2 <= c.r**2

In [13]:
# # r= 3
# # points = [Pt(*i) for i in [(1, 3), (0, 2), (4, 5), (2, 4), (0, 3)]]

# r = 5000
# points = []

# for idx, row in df.iterrows():
#     points.append(Pt(row['X_COORD'], row['Y_COORD']))

In [14]:
# n, p = len(points), points
# # All circles between two points (which can both be the same point)
# t0 = time.time()
# circles = set(sum([[c1, c2]
#                    for c1, c2 in [circles_from_p1p2r(p1, p2, r) for p1, p2 in product(p, p)]
#                    if c1 is not None], []))
# t1= time.time()
# print('circles generated: {:.2f}'.format(t1-t0))
# # points covered by each circle 
# coverage = {c: {pt for pt in points if covers(c, pt)}
#             for c in circles}
# # Ignore all but one of circles covering points covered in whole by other circles
# #print('\nwas considering %i circles' % len(coverage))
# items = sorted(coverage.items(), key=lambda keyval:len(keyval[1]))
# for i, (ci, coveri) in enumerate(items):
#     for j in range(i+1, len(items)):
#         cj, coverj = items[j]
#         if not coverj - coveri:
#             coverage[cj] = {}
# coverage = {key: val for key, val in coverage.items() if val}
# #print('Reduced to %i circles for consideration' % len(coverage))
# t1 = time.time()
# print ('circles reduced: {:.2f}'.format(t1-t0))

# # Greedy coverage choice
# chosen, covered = [], set()
# while len(covered) < n:
#     _, nxt_circle, nxt_cov = max((len(pts - covered), c, pts)
#                                  for c, pts in coverage.items())
#     delta = nxt_cov - covered
#     covered |= nxt_cov
#     chosen.append([nxt_circle, delta])

# # Output
# print('\n%i points' % n)
# print('A minimum of circles of radius %g to cover the points (And the extra points they covered)' % r)
# pp(chosen)

The above code is taking too long.

My idea is to grid LA into radius-r sized blocks, and an adjacent 'megablock', which will have a small enough n that we can calculate things sensibly.

In [15]:
# bounding coordinates based on schools

north = max(df['Y_COORD'])
south = min(df['Y_COORD'])
west = min(df['X_COORD'])
east = max(df['X_COORD'])

north, south, east, west

(4071727.5683421977,
 3989640.9564829115,
 -13152646.079689741,
 -13208547.3883814)

In [66]:
df[(df['School Level Description'] != "Elementary School") & (df['Local District'] != 'XR')].shape

(265, 17)

In [16]:
# set distance
for miles in [.75]: #reset to smallest circles
              #1, 1.25, 1.5, 1.75, 2, 2.25, 2.5, 2.75, 3]:
    
    df = df[(df['School Level Description'] != "Elementary School") & (df['Local District'] != 'XR')] #drop elementary and charter schools
    df.reset_index(inplace=True, drop=True)
    
    print(miles)
    r = int(1609.34*miles)

    # Generate Circles
    circles = set()
    for lower in range(int(south-r/2),int(north), r):
        for left in range(int(west-r/2), int(east), r):
            upper = lower+r
            right = left+r
            block_df = df.query('Y_COORD >= @lower & Y_COORD <= @upper & X_COORD >= @left & X_COORD <= @right')

            mega_lower = lower - r
            mega_upper = upper + r
            mega_left = left - r
            mega_right = right + r
            megablock_df = df.query('Y_COORD >= @mega_lower & Y_COORD <= @mega_upper & X_COORD >= @mega_left & X_COORD <= @mega_right')

            points = []
            for idx, row in block_df.iterrows():
                points.append(Pt(row['X_COORD'], row['Y_COORD']))

            mega_points = []
            for idx, row in megablock_df.iterrows():
                mega_points.append(Pt(row['X_COORD'], row['Y_COORD']))


            new_circles = set(sum([[c1, c2]
                       for c1, c2 in [circles_from_p1p2r(p1, p2, r) for p1, p2 in product(points, mega_points)]
                       if c1 is not None], []))

            circles = circles.union(new_circles)

    circle_df = pd.DataFrame(list(circles))
    print(circle_df.shape)

    # Check coverage

    arr = np.zeros([df.shape[0], circle_df.shape[0]])

    for lower in range(int(south-r/2),int(north), r):
        for left in range(int(west-r/2), int(east), r):
            upper = lower+r
            right = left+r
            block_df = df.query('Y_COORD >= @lower & Y_COORD <= @upper & X_COORD >= @left & X_COORD <= @right')

            mega_lower = lower - r
            mega_upper = upper + r
            mega_left = left - r
            mega_right = right + r
            megacircle_df = circle_df.query('y >= @mega_lower & y <= @mega_upper & x >= @mega_left & x <= @mega_right')

            for ydx, row in block_df.iterrows():
                point = Pt(row['X_COORD'], row['Y_COORD'])
                for xdx, crow in megacircle_df.iterrows():
                    circle = Cir(crow['x'], crow['y'], crow['r'])
                    if covers(circle, point):
                        arr[ydx, xdx] = 1


    keep = list()

    reduced=arr.copy()

    while reduced.sum() > 0:
        idx = reduced.sum(axis=0).argmax() #gets index of circle with most dots
        keep.append(idx)
        for xdx in np.argwhere(reduced[:,idx]==1).flatten():
            reduced[xdx,:]=0

        #print(idx, reduced.sum())

    print(len(keep))
    print(' ')

    circles_keep = circle_df.loc[keep]

    df['name'] = df['School Name']

    reset_output()

    from bokeh.tile_providers import CARTODBPOSITRON

    #changing this mask and title make it easy to make new plots
    #a little housekeeping
    reset_output()

    #add a hover tool for
    hover_tool = HoverTool(names=['Schools'],
        tooltips =[
        ('School', '@name')
    ])

    #associate the data with the plotting functions
    #mapper = LinearColorMapper(palette=pal, low=low, high=high)
    data2 = ColumnDataSource(df)

    title = 'Radius of Responsibility: {0} miles. {1} Patrols'.format(miles, len(keep))
    # and plot! center the figure in LA, set the axis to mercator coordinates
    p = figure(x_range=(la_lon-65000, la_lon+25000), y_range=(la_lat-45000, la_lat+45000),
               x_axis_type="mercator", y_axis_type="mercator",
               plot_width=900, plot_height=600, title=title)
    # add a circle for each station, sized and colored by the number of people
    p.circle(source=data2, x='X_COORD', y='Y_COORD', alpha=0.8, line_color="black",
            legend=False, name='Schools')
    #add open source geotiles
    p.add_tools(hover_tool)

    radii2 = ColumnDataSource(circles_keep)
    p.circle(source=radii2, x='x', y='y', radius='r', alpha=0.2, color='Red')

    p.add_tile(CARTODBPOSITRON)

    output_file("LAUSD_no_charter_{:.3f}.html".format(miles))
    save(p)

0.75
(781, 3)
120
 


In [20]:
circles_keep.columns

Index(['x', 'y', 'r'], dtype='object')

In [21]:
def reverse_coord_transform(row):
    x, y = pyproj.transform(google_projection,project_projection, row['x'], row['y'])
    return x, y

circles_keep['coords'] = circles_keep.apply(reverse_coord_transform, axis=1)

circles_keep[['LON', 'LAT']] = circles_keep['coords'].apply(pd.Series)

In [24]:
circles_keep.reset_index(inplace=True, drop=True)

In [26]:
circles_keep[['LON', 'LAT']].to_csv('LAUSD_no_charter_circle_locations_{:.3f}.csv'.format(miles))